# 01 - OCR Exploration

**DocSalud MX** - Exploracion del pipeline OCR para documentos medicos.

Este notebook demuestra:
1. Preprocesamiento de imagenes con OpenCV
2. Extraccion OCR con Tesseract
3. Manejo de PDFs con PyMuPDF
4. Comparacion de calidad antes/despues del preprocesamiento

In [ ]:
import sys
sys.path.insert(0, "..")

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from app.core.ocr.preprocessor import ImagePreprocessor
from app.core.ocr.extractor import OCRExtractor
from app.core.ocr.types import PreprocessConfig

%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 8)

## 1. Crear imagen sintetica de prueba

Generamos una imagen que simula un expediente medico escaneado.

In [ ]:
def create_synthetic_medical_document(width=800, height=1100):
    """Crea una imagen sintetica de documento medico."""
    img = np.ones((height, width, 3), dtype=np.uint8) * 245
    
    # Header
    cv2.putText(img, "CLINICA RURAL SAN LUIS", (150, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)
    cv2.putText(img, "Receta Medica", (280, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    cv2.line(img, (50, 120), (750, 120), (0, 0, 0), 2)
    
    # Patient data
    cv2.putText(img, "Paciente: Juan Carlos Martinez Lopez", (60, 170),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(img, "Edad: 58 anos    Sexo: M    Fecha: 15/01/2026", (60, 210),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.line(img, (50, 240), (750, 240), (150, 150, 150), 1)
    
    # Prescriptions
    cv2.putText(img, "Rx:", (60, 290),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    cv2.putText(img, "1. Metformina 850mg tabletas", (80, 340),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(img, "   1 tableta cada 12 horas por 30 dias", (80, 375),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (80, 80, 80), 1)
    cv2.putText(img, "2. Losartan 50mg tabletas", (80, 430),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(img, "   1 tableta cada 24 horas por 30 dias", (80, 465),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (80, 80, 80), 1)
    
    # Diagnosis
    cv2.line(img, (50, 520), (750, 520), (150, 150, 150), 1)
    cv2.putText(img, "Dx: Diabetes Mellitus tipo 2 (E11.9)", (60, 570),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(img, "    Hipertension arterial (I10)", (60, 610),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    
    return img

doc_img = create_synthetic_medical_document()
plt.imshow(cv2.cvtColor(doc_img, cv2.COLOR_BGR2RGB))
plt.title("Documento Medico Sintetico")
plt.axis("off")
plt.show()

## 2. Pipeline de Preprocesamiento

Visualizamos cada paso del pipeline de ImagePreprocessor.

In [ ]:
preprocessor = ImagePreprocessor()

# Pasos individuales
gray = preprocessor.to_grayscale(doc_img)
contrast = preprocessor.enhance_contrast(gray)
denoised = preprocessor.denoise(contrast)
binary = preprocessor.adaptive_threshold(denoised)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

axes[0, 0].imshow(cv2.cvtColor(doc_img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title("1. Original (BGR)")

axes[0, 1].imshow(gray, cmap="gray")
axes[0, 1].set_title("2. Escala de Grises")

axes[0, 2].imshow(contrast, cmap="gray")
axes[0, 2].set_title("3. CLAHE (Contraste)")

axes[1, 0].imshow(denoised, cmap="gray")
axes[1, 0].set_title("4. Denoised")

axes[1, 1].imshow(binary, cmap="gray")
axes[1, 1].set_title("5. Binarizado Adaptativo")

# Pipeline completo
full_pipeline = preprocessor.preprocess(doc_img)
axes[1, 2].imshow(full_pipeline, cmap="gray")
axes[1, 2].set_title("6. Pipeline Completo")

for ax in axes.flat:
    ax.axis("off")

plt.tight_layout()
plt.show()

## 3. Deteccion de Regiones de Texto

In [ ]:
regions = preprocessor.detect_text_regions(gray)

# Dibujar regiones detectadas
vis_img = doc_img.copy()
for i, (x, y, w, h) in enumerate(regions):
    cv2.rectangle(vis_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(vis_img, f"R{i}", (x, y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

plt.figure(figsize=(10, 14))
plt.imshow(cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB))
plt.title(f"Regiones de Texto Detectadas: {len(regions)}")
plt.axis("off")
plt.show()

print(f"Regiones encontradas: {len(regions)}")
for i, (x, y, w, h) in enumerate(regions):
    print(f"  Region {i}: pos=({x},{y}), size={w}x{h}")

## 4. Extraccion OCR

Ejecutamos OCR sobre el documento sintetico.

**Nota:** Requiere Tesseract instalado (`apt-get install tesseract-ocr tesseract-ocr-spa`).

In [ ]:
try:
    extractor = OCRExtractor(tesseract_lang="spa")
    result = extractor.extract_from_numpy(doc_img)
    
    print("=" * 60)
    print("RESULTADO OCR")
    print("=" * 60)
    print(f"Confianza promedio: {result.confidence:.1f}%")
    print(f"Tiempo de procesamiento: {result.processing_time_ms}ms")
    print(f"Bloques detectados: {len(result.blocks)}")
    print(f"Warnings: {result.warnings}")
    print()
    print("--- TEXTO EXTRAIDO ---")
    print(result.text)
    print("--- FIN ---")
except Exception as e:
    print(f"Tesseract no disponible: {e}")
    print("Instala con: apt-get install tesseract-ocr tesseract-ocr-spa")

## 5. Efecto del Ruido en la Calidad OCR

Comparamos la calidad OCR con y sin preprocesamiento en imagen ruidosa.

In [ ]:
# Agregar ruido a la imagen
noise = np.random.normal(0, 30, doc_img.shape).astype(np.int16)
noisy_doc = np.clip(doc_img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(18, 8))

axes[0].imshow(cv2.cvtColor(doc_img, cv2.COLOR_BGR2RGB))
axes[0].set_title("Original")

axes[1].imshow(cv2.cvtColor(noisy_doc, cv2.COLOR_BGR2RGB))
axes[1].set_title("Con Ruido (sigma=30)")

# Preprocesar imagen ruidosa
preprocessed_noisy = preprocessor.preprocess(noisy_doc)
axes[2].imshow(preprocessed_noisy, cmap="gray")
axes[2].set_title("Ruidosa + Preprocesada")

for ax in axes:
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
try:
    extractor = OCRExtractor(tesseract_lang="spa")
    
    # OCR sin preprocesamiento (directo sobre imagen ruidosa)
    import pytesseract
    raw_text = pytesseract.image_to_string(
        cv2.cvtColor(noisy_doc, cv2.COLOR_BGR2GRAY),
        config="--oem 3 --psm 6 -l spa"
    )
    
    # OCR con preprocesamiento
    result_preprocessed = extractor.extract_from_numpy(noisy_doc)
    
    print("SIN preprocesamiento:")
    print(raw_text[:300])
    print(f"\n{'='*60}")
    print(f"\nCON preprocesamiento (confianza: {result_preprocessed.confidence:.1f}%):")
    print(result_preprocessed.text[:300])
except Exception as e:
    print(f"Tesseract no disponible: {e}")

## 6. Configuracion Personalizada

El pipeline es configurable para diferentes escenarios.

In [ ]:
# Configuracion agresiva para documentos muy ruidosos
aggressive_config = PreprocessConfig(
    max_dimension=2000,
    denoise_strength=20,
    adaptive_block_size=15,
    adaptive_c=5,
    clahe_clip_limit=3.0,
)

# Configuracion suave para documentos limpios
gentle_config = PreprocessConfig(
    max_dimension=3000,
    denoise_strength=5,
    adaptive_block_size=11,
    adaptive_c=2,
    clahe_clip_limit=1.5,
)

aggressive = ImagePreprocessor(aggressive_config)
gentle = ImagePreprocessor(gentle_config)

fig, axes = plt.subplots(1, 3, figsize=(18, 8))

axes[0].imshow(preprocessor.preprocess(noisy_doc), cmap="gray")
axes[0].set_title("Default Config")

axes[1].imshow(aggressive.preprocess(noisy_doc), cmap="gray")
axes[1].set_title("Aggressive Config")

axes[2].imshow(gentle.preprocess(noisy_doc), cmap="gray")
axes[2].set_title("Gentle Config")

for ax in axes:
    ax.axis("off")

plt.tight_layout()
plt.show()

print("El pipeline es configurable segun la calidad del documento de entrada.")

## Resumen

El modulo OCR de DocSalud MX proporciona:

- **ImagePreprocessor**: Pipeline OpenCV configurable (grayscale, CLAHE, denoise, deskew, binarizacion)
- **OCRExtractor**: Motor OCR con Tesseract (imagenes) + PyMuPDF (PDFs nativos)
- **Deteccion de regiones**: Segmentacion automatica de bloques de texto
- **Metricas de calidad**: Confianza promedio y warnings automaticos

### Siguiente paso: Fase 2 - NLP y Extraccion de Entidades